In [ ]:
!pip install -q -U bitsandbytes accelerate transformers peft
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
model_path=("//content//drive//MyDrive//openhermes-mistral-7-urdu-poetry//checkpoint-2000")
tokenizer=AutoTokenizer.from_pretrained(model_path,trust_remote_code=True)
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)



BOS: <s>
EOS: </s>
PAD: <pad>
32003


In [ ]:
model_id="teknium/OpenHermes-2.5-Mistral-7B"
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
    ignore_mismatched_sizes=True,

    )
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id=tokenizer.pad_token_id
total_params=sum(p.numel() for p in model.parameters())
print(type(model))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


<class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>


In [ ]:
print(total_params)

3752095744


In [ ]:
from peft import PeftModel
model=PeftModel.from_pretrained(model,"//content//drive//MyDrive//openhermes-mistral-7-urdu-poetry//checkpoint-2000")

In [ ]:
print(type(model))
print(model.peft_config)
total_params=sum(p.numel() for p in model.parameters())
print(f"total params(base+qlora):",total_params)
trainable_params=sum(p.numel()for p in model.parameters() if p.requires_grad)
print(f"trainble params:",trainable_params)

<class 'peft.peft_model.PeftModelForCausalLM'>
{'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='teknium/OpenHermes-2.5-Mistral-7B', revision=None, inference_mode=True, r=8, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, use_qalora=False, qalora_group_size=16, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}
total params(base+qlora): 3755503616
trainble params: 0


In [ ]:
!pip install flask pyngrok
!pip install jsonify
from flask import Flask, request
from pyngrok import ngrok
import requests


  Using cached jsonify-0.5-py3-none-any.whl


In [ ]:


from flask import jsonify
app = Flask(__name__)
!ngrok config add-authtoken none of your business


def generate_reply(query,context):

    prompt = f"System: you are an urdu shayari and ghazal assistant. Here is some information that may be relevant to the user's query:\n" +f"{context.strip()}\nUser: {query.strip()}"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            repetition_penalty=1.15,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,

        )
    full_output= tokenizer.decode(outputs[0], skip_special_tokens=True)
    if full_output.startswith(prompt):
        reply_only = full_output[len(prompt):].strip()
    else:
        reply_only = full_output.strip()

    return reply_only


public_url = ngrok.connect(5000)
print("Ngrok Tunnel URL:", public_url)

@app.route("/receive", methods=["GET", "POST"])
def receive():
     data_received = request.get_json()
     if not data_received:
          return jsonify({"error": "No data found!"}), 400

     try:
        datafeed = data_received["text"]
        query = datafeed.split('response')[0].strip()
        context = datafeed.split('response')[1].strip()

        reply_from_llm = generate_reply(query, context)
        print("✅ Sent to Flask backend")
        return jsonify({"response": reply_from_llm}), 200

     except Exception as e:
          return jsonify({"error": str(e)}), 500

app.run(port=5000)




Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Ngrok Tunnel URL: NgrokTunnel: "https://a38be346e24d.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Jul/2025 20:59:18] "POST /receive HTTP/1.1" 200 -


✅ Sent to Flask backend


INFO:werkzeug:127.0.0.1 - - [16/Jul/2025 21:17:07] "POST /receive HTTP/1.1" 200 -


✅ Sent to Flask backend


...

✅ Sent to Flask backend


INFO:werkzeug:127.0.0.1 - - [16/Jul/2025 21:47:51] "POST /receive HTTP/1.1" 200 -


✅ Sent to Flask backend


INFO:werkzeug:127.0.0.1 - - [16/Jul/2025 21:59:28] "POST /receive HTTP/1.1" 200 -


✅ Sent to Flask backend
